# Overview of the Stock Data Analysis Script

This script aims to provide a comprehensive framework for performing in-depth technical analysis and graphical representation of stock data in Excel files. The following sections detail the main components of the script, function definitions, and how to process stock data files in bulk.

## Loading Modules

The script uses the following Python libraries to implement data processing, analysis, and visualization:

- **numpy**: Provides efficient array handling and mathematical operations.
- **pandas**: Used for advanced data structures and fast data analysis.
- **matplotlib**: Used to create static, interactive, and dynamic visualizations.
- **mplfinance**: Specifically used for financial data visualization, such as candlestick charts.
- **os**: Provides functionality to interact with the operating system, such as file path operations.
- **glob**: Supports Unix-style pathname pattern matching, used for file traversal.
- **plotly**: Used to create interactive graphics and data visualizations.
- **datetime**: Provides classes for handling dates and times.

## Function Definitions

### Rename DataFrame Columns

- `rename_columns(df)`: Renames the columns in the DataFrame to make them more intuitive and easy to understand, facilitating subsequent analysis.

### Create New Calculated Columns

- `add_new_columns(df)`: Adds new calculated columns to the DataFrame, such as various moving averages and interval maximum and minimum values, to support more complex technical analysis.

### Data Processing and Analysis

- `process_file(file_path)`: Reads a single Excel file, performs data preprocessing, including renaming columns, adding calculated columns, converting date formats, etc. Then, based on specific technical analysis conditions, it finds the trading days that meet the conditions and graphically displays these dates.

### Condition Filtering and Chart Drawing

- `find_dates_and_plot(df)`: Filters out trading days that meet the conditions based on set technical analysis conditions (such as price changes, trading volume, moving average relationships, etc.).

### Drawing Charts

- `plot_data(df, results, file_path)`: Draws detailed price charts and technical indicator charts for the filtered trading days to visualize the analysis results.

### Calculate Maximum Gains

- `calculate_max_gains(df, results)`: For each trading day that meets the conditions, it calculates the maximum gain within the next 250 trading days to assess the potential profitability of the strategy.

### Interactive Chart Drawing

- `plot_interactive_max_gains(...)`: Uses the plotly library to create an interactive scatter plot, showing each trading day that meets the conditions and its subsequent maximum gain, providing a more intuitive analysis perspective.

## Bulk File Processing

The script allows users to set the paths of the input and output folders, automatically finds all Excel files in the input folder, and performs the above analysis process for each file. By summarizing the analysis results of all files, the script can provide a comprehensive strategy evaluation and result visualization.

## User Guide

1. Make sure all required Python libraries are installed.
2. Place the Excel stock data files to be analyzed in the preset input folder.
3. Adjust the `data_folder_path` and `folder_path` variables at the top of the script as needed to point to the correct output and input folders.
4. Run the script to automatically process, analyze, and visualize the data.

In [1]:
# Define path variables at the top of the script

data_folder_path = '/' # Output folder

folder_path = '/' # Input folder

In [2]:
# Module loading

import numpy as np
# Basic library for numerical computation.

import pandas as pd
# Provides high-performance data structures and data analysis tools.

import matplotlib.pyplot as plt
# Library for drawing various static, interactive, and dynamic charts.

import matplotlib.dates as mdates
# Provides tools for handling date formats (such as conversion and positioning).

import matplotlib.ticker as mticker
# Used to customize the tick label positions and formats in charts.

from mplfinance.original_flavor import candlestick_ohlc
# Function specifically used for drawing candlestick charts with financial data.

import os
# Provides a way to use operating system dependent functionality.

import glob
# Supports Unix style pathname pattern expansion.

import plotly.graph_objs as go
# Object graphics library for creating interactive graphics.

from plotly.subplots import make_subplots
# Used to create composite charts containing multiple subplots.

import datetime
# Library for handling dates and times.

from matplotlib.ticker import FuncFormatter
# Allows users to customize tick label formats.

In [ ]:
# Rename DataFrame columns

def rename_columns(df):
    return df.rename(columns={
        'Dates': 'date',
        'Ticker': 'ticker',
        'Name': 'name',
        'Open Price': 'open',
        'Last Price': 'close',
        'High Price': 'high',
        'Low Price': 'low',
        'Volume': 'volume',
    })


In [ ]:
# Define new technical analysis indicators
%run add_new_columns.ipynb

In [ ]:
%run US-G.ipynb

In [ ]:
# Find stocks that meet specific technical analysis indicators
%run find_dates_and_plot.ipynb

In [ ]:
# Plotting the chart

def plot_data(df, results, file_path):
    # Set the plotting style to dark background
    plt.style.use('dark_background')

    # Iterate over all dates that meet the conditions
    for date in results:
        # Find the index of the date that meets the conditions
        index = df[df['date'] == date].index[0]
        # Calculate the start and end index of the data to be plotted
        start_index = max(index - 25, 0)  # Start 25 days before the target date
        end_index = min(index + 250, len(df))  # End 250 days after the target date
        # Extract the data to be plotted
        data_to_plot = df.iloc[start_index:end_index].copy()
        # Convert the date to matplotlib date format
        data_to_plot['date'] = mdates.date2num(data_to_plot['date'])

        # Create the plot and axes
        fig, ax = plt.subplots(figsize=(12, 6))

        # Set the chart title, including the stock code and target date
        title_name = df.loc[df['date'] == date, 'ticker'].values[0]
        title_date = date.strftime('%Y-%m-%d')
        title = f"{title_name} {title_date}"
        ax.set_title(title)

        # Plot the closing price curve
        ax.plot(data_to_plot['date'], data_to_plot['close'], color='white', lw=1, label='Close Price')

        # Plot the moving averages

        # Mark a pink diamond at the target date
        ax.scatter(mdates.date2num(date), df.loc[df['date'] == date, 'close'], color='pink', marker='d', s=50, zorder=5)

        # Add a vertical line at the target date
        ax.axvline(x=mdates.date2num(date), color='lightgray', linestyle='-', linewidth=0.5)

        # Set the grid line style
        ax.grid(color='gray', linestyle='--', linewidth=0.5)

        # Set the X-axis to date format
        ax.xaxis_date()
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

        # Set the major tick interval of the X-axis
        ax.xaxis.set_major_locator(mticker.MaxNLocator(10))

        # Automatically adjust the date display format to prevent overlap
        fig.autofmt_xdate()

        # Create a secondary axis to display the percentage change in price
        ax2 = ax.twinx()

        # Calculate and plot the percentage change in price (relative to the closing price of the target date)
        base_price = df.loc[df['date'] == date, 'close'].values[0]
        data_to_plot['pct_change'] = (data_to_plot['close'] - base_price) / base_price * 100
        ax2.plot(data_to_plot['date'], data_to_plot['pct_change'], alpha=0)

        # Set the display format of the secondary axis to percentage
        ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0f}%'.format(y)))

        # Set the tick interval of the secondary axis
        ax2.yaxis.set_major_locator(mticker.MaxNLocator(10))

        # Add a legend
        ax.legend()

        # Save the chart
        save_path = os.path.join(data_folder_path, f"{title_name}_{title_date}.png")
        plt.savefig(save_path)

        # Display the plotted chart
        #plt.show()

        # Close the chart
        plt.close(fig)

In [ ]:
# Calculate the maximum increase and the number of days to the maximum increase

def calculate_max_gains(df, results):
    # Adjustable parameters
    check_days = 250  # Number of days to check for maximum increase

    max_gains = []
    days_to_max_gain = []
    trading_days = df['date'].tolist()

    for date in results:
        condition_date_index = df.index[df['date'] == date][0]
        end_index = min(condition_date_index + check_days, len(df))  # Replace hardcoded 250 with check_days
        max_close_price = df['close'][condition_date_index:end_index].max()
        max_close_date = df['date'][df['close'][condition_date_index:end_index].idxmax()]
        gain = ((max_close_price / df.at[condition_date_index, 'close']) - 1) * 100
        trade_days_to_max = trading_days.index(max_close_date) - trading_days.index(date)

        max_gains.append(gain)
        days_to_max_gain.append(trade_days_to_max)
    
    return max_gains, days_to_max_gain

In [ ]:
# Plot an interactive chart showing the maximum increase, the number of days to the maximum increase, and the quantile line

def plot_interactive_max_gains(all_max_gains, all_days_to_max_gain, all_names, all_tickers):
    # Check if there is available data to generate the chart
    if not all_max_gains or not all_days_to_max_gain:
        print("No available data to generate interactive chart.")
        return

    # Create hover text for each data point
    hover_text = [
        f'{name} ({ticker})<br>Max Gain: {gain:.2f}%<br>Days to Max Gain: {days}'
        for name, ticker, gain, days in zip(all_names, all_tickers, all_max_gains, all_days_to_max_gain)
    ]

    # Create the chart and add scatter plot data
    fig = go.Figure(data=[go.Scatter(
        x=all_days_to_max_gain,  # X-axis: days to maximum increase
        y=all_max_gains,  # Y-axis: maximum increase percentage
        text=hover_text,  # Hover text
        mode='markers',  # Chart mode: markers
        marker=dict(
            color='yellow',  # Marker color
            size=3,  # Marker size
            line=dict(width=1, color='DarkSlateGrey'),  # Marker outline color and width
        ),
        hoverinfo='text',  # Hover info
        name='Individual Gain Points'  # Legend name
    )])

    # If there is maximum increase data, calculate and plot the 85% quantile line
    if all_max_gains:
        # Calculate the 15th percentile (i.e., data points below 85%)
        percentile_85 = np.percentile(all_max_gains, 15)
        # Create hover text showing the specific value of the percentile
        percentile_hover_text = f"Lowest 85% Gain Threshold: {percentile_85:.2f}%"
        # Add the percentile line to the chart
        fig.add_trace(go.Scatter(
            x=[min(all_days_to_max_gain), max(all_days_to_max_gain)],  # X-axis range
            y=[percentile_85, percentile_85],  # Y-axis fixed at percentile value
            mode='lines+text',  # Chart mode: lines + text
            line=dict(dash='dash', color='blue', width=1),  # Percentile line style
            name='Lowest 85% Gain Threshold',  # Legend name
            text=[percentile_hover_text] * 2,  # Set the same text at both ends
            hoverinfo='text'  # Hover info
        ))

    # Update chart layout settings
    fig.update_layout(
        title='Aggregated Max Gain After Condition Met',  # Chart title
        xaxis=dict(
            title='Trading Days to Max Gain',  # X-axis title
            showgrid=True,
            gridwidth=1,
            gridcolor='grey'
        ),
        yaxis=dict(
            title='Max Gain Percentage',  # Y-axis title
            showgrid=True,
            gridwidth=1,
            gridcolor='grey',
            range=[min(all_max_gains) - 10, max(all_max_gains) + 3],  # Y-axis range
            tickformat=',.2f%',  # Y-axis formatting
            zeroline=False,  # Do not display zero line
        ),
        plot_bgcolor='black',  # Chart background color
        paper_bgcolor='black',  # Entire drawing area background color
        font=dict(color='white'),  # Font color
        hovermode='closest',  # Hover mode
        legend=dict(title='Legend', font=dict(size=10))  # Legend settings
    )

    # Display the chart
    fig.show()

In [ ]:
# Calculate strategy outcomes, including success, failure, and not reaching the target

def calculate_strategy_outcomes(df, entry_dates):
    # Define the parameters of the strategy
    target_gain = 1.21  # The target increase is 21%
    stop_loss = 0.85    # The stop loss point is a drop of 15%
    check_days = 250   # The check period is 250 trading days

    outcomes = []  # Initialize an empty list to store the strategy results for each entry date

    # Traverse each entry date
    for entry_date in entry_dates:
        entry_index = df.index[df['date'] == entry_date][0]  # Find the index of the entry date in the DataFrame
        entry_price = df.at[entry_index, 'close']  # Get the closing price of the entry point

        # Initialize two flag variables to record whether the stop loss is triggered or the target increase is reached
        hit_stop_loss = False
        hit_target_gain = False

        # Check the subsequent check_days trading days from the entry date
        for i in range(0, min(check_days + 1, len(df) - entry_index)):
            current_price = df.at[entry_index + i, 'close']  # Get the closing price of the current check day
            if current_price <= entry_price * stop_loss:
                hit_stop_loss = True  # If the current price is below the stop loss point, record the stop loss
                break
            elif current_price >= entry_price * target_gain:
                hit_target_gain = True  # If the current price reaches the target increase, record the success
                break
        
        # Record the result of the strategy execution based on the check result
        if hit_stop_loss:
            outcomes.append((entry_date, 'Fail - Stop Loss', i, (current_price / entry_price - 1) * 100))
        elif hit_target_gain:
            outcomes.append((entry_date, 'Success', i, (current_price / entry_price - 1) * 100))
        else:
            # If neither the stop loss is triggered nor the target increase is reached within the check period, it is considered as not reaching the target
            days_to_outcome = check_days
            pct_change_at_outcome = (df.at[entry_index + check_days, 'close'] / entry_price - 1) * 100
            outcomes.append((entry_date, 'Fail - No Target Gain', days_to_outcome, pct_change_at_outcome))

    return outcomes

In [ ]:
# Plot a pie chart of the proportion of strategy success and failure

def plot_strategy_outcome_pie_chart(all_strategy_outcomes):
    """
    Use Plotly to plot a pie chart of the proportion of strategy success and failure.
    
    Parameters:
    - all_strategy_outcomes: A list containing all strategy outcomes, each element is a tuple containing the type of outcome.
    """
    # Define outcome types
    outcome_types = ['Success', 'Fail - Stop Loss', 'Fail - No Target Gain']
    # Initialize a dictionary to count the number of outcomes of each type
    outcome_counts = {outcome: 0 for outcome in outcome_types}

    # Traverse all strategy outcomes and count by outcome type
    for outcome in all_strategy_outcomes:
        outcome_counts[outcome[1]] += 1  # Increase the count of the corresponding outcome type

    # Create a pie chart with Plotly
    fig = go.Figure(data=[go.Pie(
        labels=list(outcome_counts.keys()),  # Set the labels of the pie chart to the outcome types
        values=list(outcome_counts.values()),  # Set the values of the pie chart to the counts of each type of outcome
        hole=0.3,  # Set the size of the hole in the center of the pie chart to create a donut chart effect
        marker=dict(
            colors=['#4CAF50', '#F44336', '#FFC107'],  # Customize the colors of each part of the pie chart
            line=dict(color='#FFFFFF', width=2)  # Set the color and width of the borders of each part of the pie chart
        ),
        hoverinfo='label+percent+value',  # Set the information displayed when the mouse hovers: label, percentage, and value
        textinfo='label+percent',  # Set the text information displayed on the pie chart: label and percentage
        pull=[0.1, 0, 0]  # Set the first part of the pie chart (e.g., "Success") to be slightly pulled out for emphasis
    )])

    # Update the layout settings of the pie chart
    fig.update_layout(
        title='Strategy Success vs Fail Proportion',  # Set the chart title
        plot_bgcolor='black',  # Set the chart background color
        paper_bgcolor='black',  # Set the background color of the entire drawing area
        font=dict(color='white', size=16),  # Set the font color and size
        showlegend=True,  # Show the legend
        legend=dict(
            title='Outcome',  # Set the legend title
            title_font=dict(size=15, color='white'),  # Set the font and color of the legend title
            font=dict(size=12, color='white'),  # Set the font and color of the legend items
            bgcolor='rgba(0,0,0,0)',  # Set the background color of the legend (transparent)
            bordercolor='rgba(255,255,255,0)',  # Set the border color of the legend (transparent)
            orientation="h",  # Set the legend to be placed horizontally
            x=0.5,  # Set the x position of the legend
            xanchor="center",  # Align with the center of the legend
            y=1.15  # Set the y position of the legend (above the chart)
        )
    )

    fig.show()  # Display the chart

In [ ]:
# Main function for processing the file

def process_file(file_path):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file_path)  
    # Rename the columns to a more readable format
    df = rename_columns(df)
    # Add new calculated columns, such as moving averages
    df = add_new_columns(df)
    # Convert the date column to datetime type for time series analysis
    df['date'] = pd.to_datetime(df['date'])
    # Sort the data by date
    df.sort_values('date', inplace=True)
    # Reset the index for subsequent operations
    df['ticker'] = df['ticker'].str.split('.').str[0]
    # Find the dates that meet specific technical analysis conditions
    results = find_dates_and_plot(df)
    # Plot data charts for each found date
    plot_data(df, results, file_path)
    # Output the file name and the number of results
    #print(f"File: {os.path.basename(file_path)} - Total Results: {len(results)}")
    
    # Calculate the maximum increase and the number of days required to reach the maximum increase within 250 days from the entry date
    max_gains, days_to_max_gain = calculate_max_gains(df, results)
    # Assuming the entire file only involves one stock, get the name and code of the stock
    names = [df['name'].iloc[0]] * len(results)
    tickers = [df['ticker'].iloc[0]] * len(results)

    # Calculate the strategy results for each entry date
    strategy_outcomes = calculate_strategy_outcomes(df, results)

    # Return the calculated maximum increase, the number of days required to reach the maximum increase, the stock name, the stock code, and the strategy results
    return max_gains, days_to_max_gain, names, tickers, results, strategy_outcomes

In [ ]:
# Run the script

# Initialize lists to collect data
all_max_gains = []
all_days_to_max_gain = []
all_names = []
all_tickers = []
all_dates = []
all_strategy_outcomes = []

# Find all Excel files
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# Process each found Excel file
for file_path in excel_files:
    # Process each file using the process_file function and get the relevant data
    max_gains, days_to_max_gain, names, tickers, dates, strategy_outcomes = process_file(file_path)
    
    # Accumulate the results collected from each file
    all_max_gains.extend(max_gains)
    all_days_to_max_gain.extend(days_to_max_gain)
    all_names.extend(names)
    all_tickers.extend(tickers)
    all_dates.extend(dates)  # Collect processed date data
    all_strategy_outcomes.extend(strategy_outcomes)  # Collect strategy outcome data

# Use the collected data to plot interactive charts
plot_interactive_max_gains(all_max_gains, all_days_to_max_gain, all_names, all_tickers)

# Use the collected data to plot a pie chart of the proportion of strategy success and failure
plot_strategy_outcome_pie_chart(all_strategy_outcomes)

# Process the output folder path to ensure the format is correct
if data_folder_path.endswith('/') or data_folder_path.endswith('\\'):
    data_folder_path = data_folder_path[:-1]

# Get the name of the output folder to construct the Excel file name
folder_name = os.path.basename(data_folder_path)
excel_file_name = f"{folder_name}_plot_data.xlsx"  # Construct the Excel file name
full_path = os.path.join(data_folder_path, excel_file_name)  # Full file path

# If the output folder does not exist, create it
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

# Prepare the data to be saved to Excel
data = {
    'Date': all_dates,
    'Max Gain': all_max_gains,
    'Days to Max Gain': all_days_to_max_gain,
    'Name': all_names,
    'Ticker': all_tickers,
}
df = pd.DataFrame(data)  # Create a DataFrame

# Prepare strategy outcome data, including the entry date of the strategy, the outcome, the number of days required, etc.
strategy_data = {
    'Entry Date': [outcome[0] for outcome in all_strategy_outcomes],
    'Outcome': [outcome[1] for outcome in all_strategy_outcomes],
    'Days to Outcome': [outcome[2] for outcome in all_strategy_outcomes],
    'Percentage Change at Outcome': [outcome[3] for outcome in all_strategy_outcomes],
    'Name': all_names,
    'Ticker': all_tickers,
}
df_strategy = pd.DataFrame(strategy_data)  # Create a DataFrame containing strategy outcomes

# Use Pandas Excel writer to save data to an Excel file
with pd.ExcelWriter(full_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Max Gains Data', index=False)  # Save max gains data
    df_strategy.to_excel(writer, sheet_name='Strategy Outcomes', index=False)  # Save strategy outcome data